# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Image Preprocessing Layer, Image Augmentation Layer를 통해 이미지를 변형시켜 봅시다.
            * Transfer Learning(Inception V3)
* 성능 가이드
    * Accuracy : 0.34~0.74

## 1.환경설정

* **세부 요구사항**
    - 경로 설정 : Google Colaboratory의 가상 컴퓨터 안에 데이터셋을 다운로드 받고 압축을 해제하세요.
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

### (1) 라이브러리 불러오기

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

### (2) 데이터 불러오기

#### 1) 데이터 다운로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# def download_file(file_id, save_path) :
#     if os.path.exists(save_path) :
#         print(f'{save_path} 파일이 이미 존재합니다.')
#         return

#     gdown.download(id=file_id, output=save_path, quiet=False)

In [ ]:
# file_id = '10DPW-bM7uNvX0v5rd_9PSxPc-CJX2XtZ'

# download_file(file_id, 'car_images.zip')

#### 2) 데이터 압축 해제

In [ ]:
path = "/content/drive/MyDrive/KT_aivle/CNN/mini_4/Car_Images.zip"

In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [ ]:
dataset_extract(path)

Extracting: 100%|██████████| 607/607 [00:14<00:00, 41.49files/s]


#### 2) 데이터 조회

In [ ]:
# 폴더별 이미지 데이터 갯수 확인
from glob import glob

adnormal = list(glob('/content/Car_Images/abnormal/*'))
normal = list(glob('/content/Car_Images/normal/*'))

len(normal), len(adnormal)

(302, 303)

In [ ]:
from PIL import Image

In [ ]:
# 정상 차량 랜덤 이미지 확인 및 형태 확인
image_path = '/content/Car_Images/abnormal/'

img_list = os.listdir(image_path) #디렉토리 내 모든 파일 불러오기
img_list_jpg = [img for img in img_list if img.endswith(".png")] #지정된 확장자만 필터링

print ("img_list_jpg: {}".format(img_list_jpg))

adnornal_np = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    adnornal_np.append(img_array)
    print(i, " 추가 완료 - 구조:", img_array.shape)

img_list_jpg: ['DALLíñE 2023-03-11 14.48.43 - dents of a car.png', 'DALLíñE 2023-03-11 17.14.19 - scratched car.png', 'DALLíñE 2023-03-10 18.51.26 - scratched car.png', 'DALLíñE 2023-03-11 18.43.31 - slightly dented car.png', 'DALLíñE 2023-03-11 17.18.47 - scratched car.png', 'DALLíñE 2023-03-11 17.24.02 - slightly dented car.png', 'DALLíñE 2023-03-11 17.26.14 - slightly dented car.png', 'DALLíñE 2023-03-11 17.14.17 - scratched car.png', 'DALLíñE 2023-03-11 17.26.12 - slightly dented car.png', 'DALLíñE 2023-03-10 23.58.06 - a part of car without blemish.png', 'DALLíñE 2023-03-11 01.29.03 - scratched car.png', 'DALLíñE 2023-03-11 14.52.54 - dents of a car.png', 'DALLíñE 2023-03-11 15.10.23 - dents of a car.png', 'DALLíñE 2023-03-11 01.30.49 - a little bit scratched car.png', 'DALLíñE 2023-03-11 14.50.43 - dents of a car.png', 'DALLíñE 2023-03-11 17.18.02 - scratched car.png', 'DALLíñE 2023-03-11 01.21.52 - scratched car.png', 'DALLíñE 2023-03-11 14.49.32 - dents of a car.png', 'DALLíñE 

In [ ]:
# 파손 차량 랜덤 이미지 확인 및 형태 확인

image_path = '/content/Car_Images/normal/'

img_list = os.listdir(image_path) #디렉토리 내 모든 파일 불러오기
img_list_jpg = [img for img in img_list if img.endswith(".png")] #지정된 확장자만 필터링

print ("img_list_jpg: {}".format(img_list_jpg))

nornal_np = []

for i in img_list_jpg:
    img = Image.open(image_path + i)
    img_array = np.array(img)
    nornal_np.append(img_array)
    print(i, " 추가 완료 - 구조:", img_array.shape)

img_list_jpg: ['DALLíñE 2023-03-11 14.34.13 - part of a car.png', 'DALLíñE 2023-03-11 14.44.04 - photo of part of a car.png', 'DALLíñE 2023-03-11 14.32.56 - part of a car.png', 'DALLíñE 2023-03-11 01.07.42 - photo of a part of car.png', 'DALLíñE 2023-03-11 14.35.41 - part of a car.png', 'DALLíñE 2023-03-11 00.52.55 - photo of a part of car.png', 'DALLíñE 2023-03-10 23.51.48 - a part of car without blemish.png', 'DALLíñE 2023-03-11 01.18.05 - a part of a car.png', 'DALLíñE 2023-03-11 00.25.42 - photo of a part of car without blemish.png', 'DALLíñE 2023-03-11 14.43.28 - photo of part of a car.png', 'DALLíñE 2023-03-11 01.15.21 - a part of a car.png', 'DALLíñE 2023-03-11 14.32.58 - part of a car.png', 'DALLíñE 2023-03-10 22.22.46 - photo of a part of car.png', 'DALLíñE 2023-03-11 00.51.44 - photo of a part of car.png', 'DALLíñE 2023-03-10 22.37.34 - photo of a part of car.png', 'DALLíñE 2023-03-11 01.00.12 - photo of a part of car.png', 'DALLíñE 2023-03-11 00.04.11 - a part of car without

## 2.데이터 전처리
* 모델링을 위한 데이터 구조 만들기
    * X : 이미지를 array로 변환합니다.
    * Y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.

### (1) Y : 클래스 만들기
- **세부요구사항**
    - 전체 데이터에 대한 Y를 생성합니다.
    - normal, abnormal 데이터의 수를 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:
normal_y = np.zeros(302)
normal_y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
adnormal_y = np.zeros(303)+1
adnormal_y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

### (2) X : 데이터 리스트 통합
- **세부요구사항**
    - 전체 이미지 데이터를 하나의 리스트로 통합합니다.

In [ ]:
img_data = np.concatenate((adnornal_np, nornal_np), axis=0)

img_data_y = np.concatenate((adnormal_y, normal_y), axis=0)


In [ ]:
print(img_data), print(len(img_data))

### (3) 데이터셋 분리
- **세부요구사항**
    - 데이터 스플릿의 비율 (1) : train set, test set = 90%, 10%으로 설정하세요.
    - 데이터 스플릿의 비율 (2) : train set, valid set = 90%, 10%으로 설정하세요.

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(img_data, img_data_y, test_size=.1, stratify=img_data_y)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=.1, stratify=train_y)


In [ ]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((489, 1024, 1024, 3),
 (489,),
 (55, 1024, 1024, 3),
 (55,),
 (61, 1024, 1024, 3),
 (61,))

In [ ]:
from skimage.transform import resize

# 이미지 배열의 크기를 변경하기 위한 빈 배열 생성
train_x_299 = np.zeros((489, 299, 299, 3))
val_x_299 = np.zeros((55, 299, 299, 3))
test_x_299 = np.zeros((61, 299, 299, 3))

# 각 이미지를 299x299 크기로 리사이징
for i in range(train_x.shape[0]):
    train_x_299[i] = resize(train_x[i], (299, 299, 3), anti_aliasing=True)

for i in range(val_x.shape[0]):
    val_x_299[i] = resize(val_x[i], (299, 299, 3), anti_aliasing=True)

for i in range(test_x.shape[0]):
    test_x_299[i] = resize(test_x[i], (299, 299, 3), anti_aliasing=True)


In [ ]:

train_x_r = train_x_299[:,:,:,0] / 255.
train_x_g = train_x_299[:,:,:,1] / 255.
train_x_b = train_x_299[:,:,:,2] / 255.

train_x_s = np.stack ((train_x_r, train_x_g ,train_x_b), axis=3)

val_x_r = val_x_299[:,:,:,0] / 255.
val_x_g = val_x_299[:,:,:,1] / 255.
val_x_b = val_x_299[:,:,:,2] / 255.

val_x_s = np.stack ((val_x_r, val_x_g ,val_x_b), axis=3)

test_x_r = test_x_299[:,:,:,0] / 255.
test_x_g = test_x_299[:,:,:,1] / 255.
test_x_b = test_x_299[:,:,:,2] / 255.

test_x_s = np.stack ((test_x_r, test_x_g ,test_x_b), axis=3)

train_x_s.shape, test_x_s.shape, val_x_s.shape

((489, 299, 299, 3), (61, 299, 299, 3), (55, 299, 299, 3))

### (4) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다.
    * 이미지 파일을 불러와 Training data, Validation data, Test data 각각 array 형태로 변환해 봅시다.

## 3.모델링 I
* **세부요구사항**
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

### (1) 모델 1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

In [ ]:
import keras
from keras.utils import clear_session
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, RandomTranslation, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.losses import binary_crossentropy

#### 1) 구조 설계

## 모델1

In [ ]:
clear_session()

il = Input(shape=(299,299,3))
al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
hl = Conv2D(8,kernel_size=(5,5),activation='relu')(al)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(32, kernel_size=(5,5),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(128, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(256, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
# hl = Conv2D(512, kernel_size=(5,5),activation='relu')(hl)
# hl = BatchNormalization()(hl)
# hl = Dropout(0.2)(hl)
hl = Flatten()(hl)
hl = Dense(1024, activation='relu')(hl)
hl = Dense(128, activation='relu')(hl)
ol = Dense(1, activation='sigmoid')(hl)

model = Model(il,ol)

model.summary()

model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 299, 299, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 299, 299, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 295, 295, 8)         │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 147, 147, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 147, 147, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 147, 147, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 143, 143, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 71, 71, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 71, 71, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 71, 71, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 23, 23, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 11, 11, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 11, 11, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 11, 11, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 3, 3, 256)           │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │              

 Total params: 1,325,217 (5.06 MB)

 Trainable params: 1,324,369 (5.05 MB)

 Non-trainable params: 848 (3.31 KB)

#### 2) 학습

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 30,
    min_delta= 0,
    restore_best_weights= True
)

In [ ]:
hist = model.fit(train_x_s, train_y, epochs=10000, validation_split=.2,callbacks = [es])

Epoch 1/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.7593 - loss: 0.4808 - val_accuracy: 0.5204 - val_loss: 0.7230
Epoch 2/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.7881 - loss: 0.4772 - val_accuracy: 0.5204 - val_loss: 0.6929
Epoch 3/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.7739 - loss: 0.5082 - val_accuracy: 0.5204 - val_loss: 1.0815
Epoch 4/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.7804 - loss: 0.4279 - val_accuracy: 0.5204 - val_loss: 1.1372
Epoch 5/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.8819 - loss: 0.3480 - val_accuracy: 0.5204 - val_loss: 1.0580
Epoch 6/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8543 - loss: 0.3865 - val_accuracy: 0.5204 - val_loss: 1.1138
Epoch 7/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.7895 - loss: 0.4279 - val_accuracy: 0.5204 - val_loss: 1.3670
Epoch 8/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.8500 - loss: 0.3986 - 

#### 3) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
from sklearn.metrics import *
pred_val = model.predict(val_x_s)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
              precision    recall  f1-score   support

         0.0       0.81      0.78      0.79        27
         1.0       0.79      0.82      0.81        28

    accuracy                           0.80        55
   macro avg       0.80      0.80      0.80        55
weighted avg       0.80      0.80      0.80        55



In [ ]:
pred_val = model.predict(test_x_s)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 654ms/step
              precision    recall  f1-score   support

         0.0       0.92      0.73      0.81        30
         1.0       0.78      0.94      0.85        31

    accuracy                           0.84        61
   macro avg       0.85      0.83      0.83        61
weighted avg       0.85      0.84      0.83        61



## 모델2

In [ ]:
clear_session()

il = Input(shape=(299,299,3))
al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
hl = Conv2D(8,kernel_size=(5,5),padding='same',activation='relu')(al)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(32, kernel_size=(5,5),padding='same',activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(3,3))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(128, kernel_size=(5,5),padding='same',strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(256, kernel_size=(5,5),padding='same',strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
# hl = Conv2D(512, kernel_size=(5,5),padding='same',activation='relu')(hl)
# hl = MaxPool2D(pool_size=(2,2),strides=(2,2))(hl)
# hl = BatchNormalization()(hl)
# hl = Dropout(0.2)(hl)
hl = Flatten()(hl)
hl = Dense(1024, activation='relu')(hl)
hl = Dense(128, activation='relu')(hl)
ol = Dense(1, activation='sigmoid')(hl)

model = Model(il,ol)

model.summary()

model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 299, 299, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 299, 299, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 299, 299, 8)         │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 149, 149, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 149, 149, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 149, 149, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 149, 149, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 49, 49, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 49, 49, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 49, 49, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 17, 17, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 3, 3, 256)           │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │              

 Total params: 1,325,217 (5.06 MB)

 Trainable params: 1,324,369 (5.05 MB)

 Non-trainable params: 848 (3.31 KB)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 70,
    min_delta= 0,
    restore_best_weights= True
)

In [ ]:
hist = model.fit(train_x_s, train_y, epochs=10000, validation_split=.2,callbacks = [es])

Epoch 1/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 197ms/step - accuracy: 0.6302 - loss: 0.6908 - val_accuracy: 0.5204 - val_loss: 0.7012
Epoch 2/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.7023 - loss: 0.5759 - val_accuracy: 0.5204 - val_loss: 0.7506
Epoch 3/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.7688 - loss: 0.5273 - val_accuracy: 0.5204 - val_loss: 0.7298
Epoch 4/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.8024 - loss: 0.4404 - val_accuracy: 0.5204 - val_loss: 1.1443
Epoch 5/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.8009 - loss: 0.4268 - val_accuracy: 0.5204 - val_loss: 1.0344
Epoch 6/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.8324 - loss: 0.3795 - val_accuracy: 0.5204 - val_loss: 1.2874
Epoch 7/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.8184 - loss: 0.4155 - val_accuracy: 0.5204 - val_loss: 1.7770
Epoch 8/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.8233 - loss: 0.4097 -

In [ ]:
from sklearn.metrics import *
pred_val = model.predict(val_x_s)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step
              precision    recall  f1-score   support

         0.0       0.88      0.78      0.82        27
         1.0       0.81      0.89      0.85        28

    accuracy                           0.84        55
   macro avg       0.84      0.84      0.84        55
weighted avg       0.84      0.84      0.84        55



In [ ]:
pred_val = model.predict(test_x_s)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step
              precision    recall  f1-score   support

         0.0       0.81      0.73      0.77        30
         1.0       0.76      0.84      0.80        31

    accuracy                           0.79        61
   macro avg       0.79      0.79      0.79        61
weighted avg       0.79      0.79      0.79        61



## 모델3

In [ ]:
clear_session()

il = Input(shape=(299,299,3))
al = RandomTranslation(height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3))(il)
hl = Conv2D(8,kernel_size=(5,5),activation='relu')(al)
hl = Conv2D(8,kernel_size=(5,5),activation='relu')(al)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(32, kernel_size=(5,5),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(128, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Conv2D(256, kernel_size=(5,5),strides=(3,3),activation='relu')(hl)
hl = MaxPool2D(pool_size=(3,3),strides=(2,2))(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)
hl = Flatten()(hl)
hl = Dense(1024, activation='relu')(hl)
hl = Dense(128, activation='relu')(hl)
ol = Dense(1, activation='sigmoid')(hl)

model = Model(il,ol)

model.summary()

model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 299, 299, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation                   │ (None, 299, 299, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 295, 295, 8)         │             608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 147, 147, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 147, 147, 8)         │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 147, 147, 8)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 143, 143, 32)        │           6,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 71, 71, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 71, 71, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 71, 71, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 23, 23, 128)         │         102,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 11, 11, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 11, 11, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 11, 11, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 3, 3, 256)           │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │              

 Total params: 1,325,217 (5.06 MB)

 Trainable params: 1,324,369 (5.05 MB)

 Non-trainable params: 848 (3.31 KB)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 70,
    min_delta= 0,
    restore_best_weights= True
)

In [ ]:
hist = model.fit(train_x_s, train_y, epochs=10000, validation_split=.2,callbacks = [es])

Epoch 1/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.8054 - loss: 0.4586 - val_accuracy: 0.5204 - val_loss: 0.7923
Epoch 2/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.7862 - loss: 0.4784 - val_accuracy: 0.5204 - val_loss: 0.8841
Epoch 3/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.8333 - loss: 0.3959 - val_accuracy: 0.5204 - val_loss: 1.3927
Epoch 4/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.7862 - loss: 0.4616 - val_accuracy: 0.5204 - val_loss: 1.2247
Epoch 5/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.7876 - loss: 0.4471 - val_accuracy: 0.5204 - val_loss: 1.5362
Epoch 6/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8476 - loss: 0.4014 - val_accuracy: 0.5204 - val_loss: 2.0840
Epoch 7/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8562 - loss: 0.3568 - val_accuracy: 0.5204 - val_loss: 2.7582
Epoch 8/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8214 - loss: 0.4054 - 

In [ ]:
from sklearn.metrics import *
pred_val = model.predict(val_x_s)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86        27
         1.0       0.88      0.82      0.85        28

    accuracy                           0.85        55
   macro avg       0.86      0.86      0.85        55
weighted avg       0.86      0.85      0.85        55



In [ ]:
pred_val = model.predict(test_x_s)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.70      0.82        30
         1.0       0.78      1.00      0.87        31

    accuracy                           0.85        61
   macro avg       0.89      0.85      0.85        61
weighted avg       0.89      0.85      0.85        61



## 4.모델링 II
* **세부요구사항**
    - image_dataset_from_directory를 이용하여 training set, validation set을 만들어봅시다.
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Image Preprocessing Layer, Image Augmentation Layer를 통해 이미지를 변형시켜 봅시다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.

### (1) image_dataset_from_directory

### (2) Transfer Learning

#### 1) Inception V3 모델을 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 False로 설정하여 불러온 모델의 모든 레이어들을 Frozen 시켜 가중치를 고정합니다.

In [ ]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((489, 1024, 1024, 3),
 (489,),
 (55, 1024, 1024, 3),
 (55,),
 (61, 1024, 1024, 3),
 (61,))

# data resize

((489, 299, 299, 3), (55, 299, 299, 3), (61, 299, 299, 3))

In [ ]:
clear_session()

base_model = keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape= (299,299,3)
    )

new_output = GlobalAveragePooling2D()(base_model.output)
hl = Dropout(.3)(new_output)
new_output = Dense(1024, activation = 'relu')(hl)
new_output = Dense(1, activation = 'sigmoid')(new_output)


model = Model(base_model.inputs, new_output)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 299, 299, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 149, 149, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 149, 149, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 149, 149, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 147, 147, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 147, 147, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 147, 147, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 147, 147, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 147, 147, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 147, 147, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 73, 73, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 73, 73, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 73, 73, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 73, 73, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 71, 71, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 71, 71, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 23,901,985 (91.18 MB)

 Trainable params: 23,867,553 (91.05 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [ ]:
model.compile(optimizer='adam', loss = binary_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(
    monitor = 'val_loss',
    patience = 70,
    min_delta= 0,
    restore_best_weights= True
)

In [ ]:
hist = model.fit(train_x_s, train_y, epochs=10000, validation_split=.2,callbacks = [es])

Epoch 1/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - accuracy: 0.6489 - loss: 1.0571 - val_accuracy: 0.4796 - val_loss: 8.2779
Epoch 2/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 402ms/step - accuracy: 0.9628 - loss: 0.1371 - val_accuracy: 0.4796 - val_loss: 4.3909
Epoch 3/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 406ms/step - accuracy: 0.9482 - loss: 0.2122 - val_accuracy: 0.5204 - val_loss: 2.0685
Epoch 4/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 396ms/step - accuracy: 0.9316 - loss: 0.1640 - val_accuracy: 0.4796 - val_loss: 4.0957
Epoch 5/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 398ms/step - accuracy: 0.9491 - loss: 0.1163 - val_accuracy: 0.4796 - val_loss: 2.8311
Epoch 6/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 400ms/step - accuracy: 0.9616 - loss: 0.1437 - val_accuracy: 0.4796 - val_loss: 2.7965
Epoch 7/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 402ms/step - accuracy: 0.9248 - loss: 0.1420 - val_accuracy: 0.4796 - val_loss: 914.3773
Epoch 8/10000
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 400ms/step - accuracy: 0.9279 - loss: 

In [ ]:
from sklearn.metrics import *
pred_val = model.predict(val_x_s)
pred_val = np.where(pred_val>0.5, 1, 0)
print(classification_report(val_y, pred_val))

2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step
              precision    recall  f1-score   support

         0.0       0.96      0.81      0.88        27
         1.0       0.84      0.96      0.90        28

    accuracy                           0.89        55
   macro avg       0.90      0.89      0.89        55
weighted avg       0.90      0.89      0.89        55



In [ ]:
pred_val = model.predict(test_x_s)
pred_test = np.where(pred_val>0.5, 1, 0)
print(classification_report(test_y, pred_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
              precision    recall  f1-score   support

         0.0       0.83      0.80      0.81        30
         1.0       0.81      0.84      0.83        31

    accuracy                           0.82        61
   macro avg       0.82      0.82      0.82        61
weighted avg       0.82      0.82      0.82        61



#### 2) Inception V3 모델과 연결하여 구조 설계
* Inception V3을 불러온 후, 다른 레이어와 연결하기
* Image Preprocessing Layer, Image Augmentation Layer를 사용해보세요.

In [ ]:
model.save('best.keras')

#### 3) 학습

#### 4) 성능 평가